In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot



In [ ]:
data_train = pd.read_csv('/content/train.csv')
data_train.head()

ID  ... Sub-task B
0  C45.451  ...       NGEN
1   C47.11  ...       NGEN
2   C33.79  ...       NGEN
3  C4.1961  ...       NGEN
4  C10.153  ...       NGEN

[5 rows x 4 columns]

In [ ]:

data_test = pd.read_csv('/content/test.csv')


In [ ]:
data_train.drop(['ID','Sub-task B'], axis=1, inplace=True)
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)




In [ ]:
NAG_train = data_train[(data_train['Sub-task A']=='NAG')]
OAG_train = data_train[(data_train['Sub-task A']=='CAG')]
CAG_train = data_train[(data_train['Sub-task A']=='OAG')]

In [ ]:
NAG_test = data_test[(data_test['Sub-task A']=='NAG')]
OAG_test= data_test[(data_test['Sub-task A']=='CAG')]
CAG_test = data_test[(data_test['Sub-task A']=='OAG')]

In [ ]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_train["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_train["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_train["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

#NAG_train.head()

In [ ]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_test["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_test["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_test["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_test.head()

Text  Sub-task A
0        U deserve more subscribers. U really great.           0
1                                     Nice video....           0
2  sorry if i bother somebody.. iam a defence asp...           0
3  Joker was amazing....it was not glamorised !.....           0
4                                          Nice baro           0

In [ ]:
data_train = pd.concat([NAG_train,
                 OAG_train,
                  CAG_train
                ],axis=0)

data_train.reset_index(inplace=True)

In [ ]:
data_test = pd.concat([NAG_test,
                  OAG_test,
                  CAG_test
                 ],axis=0)

data_test.reset_index(inplace=True)

In [ ]:

data_test = data_test.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [ ]:

data_train = data_train.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [ ]:
data_train

index                                               Text  label
0         0                                          Next part      0
1         1                 Iii8mllllllm\nMdxfvb8o90lplppi0005      0
2         2  🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...      0
3         3  What the fuck was this? I respect shwetabh and...      0
4         4  Concerned authorities should bring arundathi R...      0
...     ...                                                ...    ...
4258   4238  Personality of a beggar lasts with her/him eve...      2
4259   4245  Ghanta..ek no ek chutiya galti se iski video l...      2
4260   4253           Ban suedo femists and liberals ...F..off      2
4261   4254                                Open bob and vagane      2
4262   4258  Abey loudey Arnab... Did u ever see the vedios...      2

[4263 rows x 3 columns]

In [ ]:
data_train.drop(['index'], axis=1, inplace=True)

In [ ]:
X = data_train.Text
Y = data_train.label
le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X,Y,test_size=0.15)

In [ ]:

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(len(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(len(y_valid)))

Train data len:3623
Class distribution3623
Valid data len:640
Class distribution640


In [ ]:
num_words = 50000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(X_train)


print(str(tokenizer.texts_to_sequences(['xyz how are you'])))

[[3079, 74, 12, 9]]


In [ ]:

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )



x_train = pad_sequences(x_train, padding='post', maxlen=150)
x_valid = pad_sequences(x_valid, padding='post', maxlen=150)
x_test = pad_sequences(x_test, padding='post', maxlen=150)



train_labels = np.asarray(y_train )
valid_labels = np.asarray( y_valid)

test_labels = np.asarray(data_test['label'].tolist())

#print('Train data len:'+str(len(x_train)))
#print('Class distribution'+str(Counter(train_labels)))

#print('Validation data len:'+str(len(x_valid)))
#print('Class distribution'+str(Counter(valid_labels)))

#print('Test data len:'+str(len(x_test)))
#print('Class distribution'+str(Counter(test_labels)))


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

In [ ]:

max_features =50000
embedding_dim =16
sequence_length = 150


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 64, input_length=sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 30
history_bi = model.fit(x_train,train_labels, epochs=num_epochs, validation_data=(x_valid, valid_labels))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 64)           3200000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 150, 128)          66048     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 3,311,489
Trainable params: 3,311,489
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
114/114 [==============================] - 40s 301ms/step - loss: 0.0000e+00 - accuracy: 0.7850 - v

In [ ]:
#plt.plot(history.model['loss'], label=' training data')
#plt.plot(history.history['val_loss'], label='validation data')
#plt.title('Loss for Text Classification')
#plt.ylabel('Loss value')
#plt.xlabel('No. epoch')
#plt.legend(loc="upper left")
#plt.show()

In [ ]:

x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=150)


In [ ]:
print("Generate predictions for all samples")
predictions = model.predict(x_test)

Generate predictions for all samples


In [ ]:

data_test['pred_sentiment']= predictions

In [ ]:

labels = [0,1,2]
    
print(classification_report(data_test['label'].tolist(),data_test['pred_sentiment'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82       836
           1       0.09      0.12      0.10       117
           2       0.00      0.00      0.00       113

    accuracy                           0.68      1066
   macro avg       0.29      0.32      0.31      1066
weighted avg       0.63      0.68      0.65      1066

